# Summing over subsets of size K

Here we give a simple model of subsets of $K$ from a base set $\mathcal{Y} = \{1, \ldots, N\}$.  We assume that each element $i \in \mathcal{Y}$ has a weight $w_i$.

The probability of a set $Y \subseteq \mathcal{Y}$ is

$$
p(Y) \propto \begin{cases}
\prod_{i \in Y} w_i & \textbf{if } |Y|=K \\
0 & \textbf{otherwise}
\end{cases}
$$

We can compute the normalization constant of this distribution using a generalized version of the dynamic programming algorithm for computing [binomial coefficients](https://en.wikipedia.org/wiki/Binomial_coefficient).

In [5]:
import numpy as np
from hypergraphs.semirings import LazySort, flatten

def subsets(w, K, zero, one):
    "Subsets of size K"
    N = len(w)
    E = np.full((K+1,N+1), zero)
    E[0,:] = one                     # initialization
    for k in range(1, K+1):
        for n in range(N):
            E[k,n+1] = E[k,n] + E[k-1,n] * w[n]
    return E[K,N]

Below, we use the `LazySort` semiring to see all of the structures that we summed over in order of the highest scoring subsets.

In [24]:
N = 6
K = 3 
ws = np.random.uniform(0, 1, size=N)
z = subsets([LazySort(w, i) for i, w in enumerate(ws)], K, LazySort.zero(), LazySort.one())
Z = subsets(ws, K, 0.0, 1.0)  # normalization constant computed in the sum-product semiring
for x in z:
    print(f'{x.score/Z:.3f} {set(flatten(x.data))}')  # print subsets sorted by decreasing probability

0.107 {2, 3, 4}
0.094 {1, 3, 4}
0.080 {1, 2, 3}
0.074 {3, 4, 5}
0.063 {2, 3, 5}
0.060 {1, 2, 4}
0.057 {0, 3, 4}
0.055 {1, 3, 5}
0.048 {0, 2, 3}
0.047 {2, 4, 5}
0.042 {0, 1, 3}
0.042 {1, 4, 5}
0.036 {0, 2, 4}
0.035 {1, 2, 5}
0.033 {0, 3, 5}
0.032 {0, 1, 4}
0.027 {0, 1, 2}
0.025 {0, 4, 5}
0.021 {0, 2, 5}
0.019 {0, 1, 5}


As usual, we can use the outside algorithm to compute marginal sums.

In [38]:
# TODO: It's annoying to write the computation in two ways.  It would be much nicer 
# to extract the hypergraph by running the dynamic program with its loops and all.
# I suppose we could do that by a "hypergraph semiring" which accumulates Edge objects 
# creates.  However, it's unclear how the nodes and edges get named.  I think the answer 
# to that is that the memo table will name nodes by a the key used.  This seems ok: 
#     chart[key]            +=             chart[body1]    *     ....   *   chart[bodyK] 
#      ^^^^^^^              ^^                            ^^^          ^^^
#    create/lookup node    add hyperedge           hyperedge body is a K-tuple 

from hypergraphs import Hypergraph
def subset_graph(w, K):
    "Subsets of size K"
    N = len(w)
    G = Hypergraph(root=(K,N))
    for n in range(N):
        G.edge(1.0, (0,n))        # terminal
    for k in range(1, K+1):
        for n in range(N):
            G.edge(1.0, (k,n+1), (k,n))
            G.edge(w[n], (k,n+1), (k-1,n))  # we, can alternatively make w[n] a node, 
                                            # which will make the outside score equal to incl/w.
    return G

G = subset_graph(ws, K)
#G.show()